В современном глубоком обучении под понятием "deconvolution" обычно понимают транспонированную свертку (см. [A guide to convolution arithmetic for deep learning]($A guide to convolution arithmetic for deep learning$)). Но данная работа, написанная в 2010 году, не имеет отношения к транспонированной свертке. Авторы пробуют ответить на следующий вопрос: как изображение можно представить в виде результата свертки?

### Неформальная постановка задачи

Рассмотрим сверточную сеть, которая отображает карту признаков $M$ в изображение $I$. Имея датасет из изображений, как подобрать веса сети, общие для всех изображений, таким образом, чтобы для каждого изображения существовала карта признаков, отображающаяся сетью в исходное изображение с минимальной погрешностью?  И если веса сети найдены, то как для изображений найти искомые карты признаков?

Решив эту задачу, найденные карты признаков можно будет использовать для классификации изображений, применяя методы компьютерного зрения не к исходному изображению, а к его карте признаков. Предполагается, что по карте признаков классификацию осуществлять будет легче, чем по исходному изображению, потому что карта признаков более явно описывает ключевые детали на изображении (например, границы объектов, окна дома, пальцы человека, полосы на арбузе и пр.), которые затем отображаются сверточной сетью в исходное изображение.

### Формальная постановка задачи

**Сверточный слой**

В работе используются такие же сверточные слои, как в [LeNet]($Gradient-Based Learning Applied to Document Recognition$) и во многих современных сверточных сетях. Они имеют много фильтров и обрабатывают карты признаков со многими каналами. Формально работу сверточного слоя можно записать следующим образом:

$\text{output}_c = \sum\limits_{k=1}^K \text{input}_k \oplus W_{k, c}$

Здесь $\text{output}_c$ - выходной канал с индексом $c$, $\text{input}_k$ - входной канал с индексом $k$.

Обычно фильтры плотно связаны друг с другом, то есть каждый входной канал влияет на каждый выходной канал. Однако можно рассмотреть и разреженные сверточные слои, в которых лишь некоторые фильтры обрабатывают некоторые входные каналы. Чтобы добиться такого свойства, мы можем умножать результат свертки на бинарную матрицу $g$ (из нулей и единиц), в которой единица означает связь i-го входного канала с j-м выходным каналом:

$\text{output}_c = \sum\limits_{k=1}^K g_{k, c} \big(\text{input}_k \oplus W_{k, c}\big)$

Либо мы можем занулять некоторые из весов $W_{k, c}$, что даст тот же результат.

Далее для краткости сверточный слой будем записывать как $\text{Conv}\ (\text{input}, W, g)$.

**Сверточная сеть**

Авторы описывают сеть из последовательности сверточных слоев. Таким образом, в ней нет нелинейностей, прибавления bias'ов, операций pooling и upsampling. Например, сеть из трех слоев может быть записана таким образом:

$\text{output} = \text{Conv} (\text{Conv} (\text{Conv} (\text{input}, W_1, g_1), W_2, g_2), W_3, g_3)$

Такая сеть отображает карту признаков $M$ в изображение $I$, используя веса $W = (W_1, W_2, W_3)$ и бинарные матрицы связности $g = (g_1, g_2, g_3)$:

$f: M \times W \times g \to I$.

**Поиск операции, обратной свертке**

Поиск операции, обратной свертке, означает решение задачи $Y = \text{Conv}\ (X, W)$ относительно $X$. Сверточный слой, согласно авторам работы, не является однозначно обратимым, то есть решение не единственно. Авторы рассматривают данную задачу как задачу оптимизации: ее можно решать, например, градиентным спуском. Чтобы решение не расходилось (элементы $X$ не стремились к $\pm\infty$) на $X$ дополнительно навешивается функция потерь, равная сумме $p$-х степеней модулей всех элементов $X$. Если $p=1$, то это L1-регуляризация массива $X$. Далее опишем более формально.

**Формулировка задачи оптимизации**

Сформулируем задачу в том виде, в каком ее описывают авторы.

Сеть $f$ должна восстанавливать изображение из его карты признаков. Восстановленное изображение мы можем сравнить с исходным с помощью среднеквадратичного отклонения. И восстановленное, и исходное изображение являются трехмерными массивами $H \times W \times C$. Среднеквадратичное отклонение между исходным и восстановленным изображением можно записать таким образом:

$\text{MSE} (A, B) = \sum_{i, j, k} (A_{i, j, k} - B_{i, j, k})^2$

L1-регуляризацию карты признаков $M$ можно описать, введя следущую функцию потерь:

$\text{L}_1(M) = \sum_{i, j, k} |M_{i, j, k}|$

Пусть мы имеем сверточную сеть $f$ с набором весов $W$ и набором матриц связности $g$, а также имеем одно изображение $\textit{image}$. Обозначим его карту признаков за $\textit{map}$ и запишем выражение для функции потерь, с учетом L1-регуляризации карты признаков $M$:

$\text{loss} (\textit{image}, \textit{map}, W, g) = \text{MSE}(\textit{image}, f(\textit{map}, W, g)) + \lambda * \text{L}_1(\textit{map})$

Соответственно, $\text{loss}$ по всему датасету из изображений $\{\textit{image}_i\}_{i=1}^N$ - это $\text{loss}$, усредненный по всем изображениям. 

*Примечание: Запись выше соответствует формуле (3) из раздела 3.1 статьи. Формула в статье дана для сети, состоящей из 2 сверточных слоев. Матрица связаности второго (выходного) слоя состоит из единиц и опускается в записи формулы (3).*

Изображение и матрица связности даны заранее, и нужно решать эту задачу оптимизации одновременно относительно весов сети $W$ (общих для всех изображений) и карт признаков для каждого изображения $\{\textit{map}_i\}_{i=1}^N$.

- Решая задачу относительно $W$, мы находим оптимальные веса сети, для которых существуют "точные" карты признаков (то есть отображающиеся в исходное изображение с минимальной погрешностью).

- Решая задачу относительно $\{\textit{map}_i\}_{i=1}^N$, мы находим эти карты признаков, используя веса сети.

После завершения оптимизации мы фиксируем веса $W$, и теперь можем получать карты признаков для произвольного изображения, решая задачу оптимизации относительно карты признаков этого изображения.

**Решение задачи оптимизации**

Авторы анализируют полученную задачу оптимизации и используют трюк с введением дополнительной переменной. Не будем здесь подробно на этом останавливаться (см. раздел 3.2 статьи). Кратко опишем, что основная суть трюка заключается в следующем: вводятся дополнительные массивы $X$ и вместо минимизации $\text{L}_1(\textit{map})$ попеременно минимизируются $\text{L}_1(\textit{X})$ и среднеквадратичное отклонение $X$ от $\textit{map}$.

Обучаемые переменные (веса сети и карты признаков) случайно инициализируются нормальным распределением и оптимизируются градиентным спуском.

### Результаты

Авторы обучают описанную модель на двух датасетах: содержащих изображения фруктов и зданий.

Используется три сверточных слоя. Третий (выходной) слой имеет матрицу $g$, состоящую из единиц, и один фильтр, соответствующий одному каналу выходного изображения. Первый и второй сверточные слои разрежены. Некоторые фильтры обрабатывают два канала, некоторые - один канал.

Сверточные фильтры, полученные в результате обучения, показаны на иллюстрации.

На иллюстрации показано 9 фильтров в первом слое, хотя выше говорилось про один фильтр. Дело в том, что в современном компьютерном зрении и в данном обзоре под фильтром обычно понимается многоканальный фильтр, который представляет собой набор черно-белых фильтров, а в статье под фильтром понимается черно-белый фильтр.

<img src="assets/deconv.jpg" width="700" align="center">

### Резюме

В статье представлен способ обучения на неразмеченных изображениях (unsupervised learning), при котором **изображения из датасета представляются в виде результатов работы сверточной сети**.

**Обучение** на датасете $\{\textit{X}_i\}_{i=1}^N$ заключается в том, что одновременно оптимизируются:

1. Веса сверточной сети $W$.
2. Карты признаков для каждого изображения $\{\textit{M}_i\}_{i=1}^N$

Целевой метрикой является среднеквадратичное отклонение исходных изображений $X_i$ от воccтановленных по картам признаков. Также на карты признаков навешивается L1-регуляризация. После обучения мы получаем обученные веса $W$ сверточной сети.Сверточная сеть при этом является разреженной, то есть отдельные фильтры принимают на вход лишь отдельные каналы.

**Инференс** также является процессом оптимизации, в ходе которого для заданного изображения и весов сети ищется карта признаков, по которой можно восстановить исходное изображение с минимальной погрешностью.

Аналогичный процесс может быть применен для того, чтобы попробовать обратить любую операцию свертки, то есть зная выходные данные и веса попробовать найти входные данные. Но для большинства фильтров (например, фильтр обесцвечивания или выделения границ) такое решение будет не единственно.

В последующих работах ([1]($Adaptive deconvolutional networks for mid and high level feature learning$), [2]($Visualizing and Understanding Convolutional Networks$)) авторы продолжают рассматривать сверточные сети, в которых информация может распространяться в двух направлениях, и дополняют сети операцией транспонированной свертки.